In [8]:
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build

# ✅ 1. 환경변수 로드 (현재 경로의 .env 사용)
load_dotenv(dotenv_path=r"c:\\Users\\NOTE2\\조원C&I\\개인biz\\phytho code\\youtube analytics\\.env")
API_KEY = os.getenv('YOUTUBE_API_KEY')

# ✅ 2. API 클라이언트 설정
youtube = build('youtube', 'v3', developerKey=API_KEY)

# ✅ 3. 영상 ID 검색 (검색어 기반)
def get_video_ids_by_keyword(keyword, max_results=50):
    request = youtube.search().list(
        q=keyword,
        part='id',
        type='video',
        maxResults=max_results,
        order='date'
    )
    response = request.execute()
    return [item['id']['videoId'] for item in response['items']]

# ✅ 4. 영상 상세 정보 조회 (50개까지 batch 조회)
def get_video_details(video_ids):
    video_data = []
    for i in range(0, len(video_ids), 50):
        ids = video_ids[i:i+50]
        request = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=','.join(ids)
        )
        response = request.execute()
        for item in response['items']:
            stats = item.get('statistics', {})
            snippet = item['snippet']
            video_data.append({
                'videoId': item['id'],
                'title': snippet['title'],
                'channel': snippet['channelTitle'],
                'pub_date': snippet['publishedAt'],
                'counts': stats.get('viewCount', '0'),
                'likes_count': stats.get('likeCount', '0'),
                'comment_count': stats.get('commentCount', '0'),
                'url': f"https://www.youtube.com/watch?v={item['id']}"
            })
    return video_data

# ✅ 5. 실행 및 저장
if __name__ == '__main__':
    keyword = "이재명 전남 방문"  # 여기에 분석할 키워드 입력
    video_ids = get_video_ids_by_keyword(keyword, max_results=50)
    video_details = get_video_details(video_ids)

    df = pd.DataFrame(video_details)

    # ✅ 미리보기 출력
    print("\n📋 영상 수집 미리보기 (상위 5개):")
    print(df[['title', 'channel', 'pub_date', 'url']].head(5))

    output_file = f'youtube_metadata_{keyword}.csv'.replace(' ', '_')
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"\n✅ '{keyword}'에 대한 영상 {len(df)}건 저장 완료 → {output_file}")



📋 영상 수집 미리보기 (상위 5개):
                                               title   channel  \
0                        민주당 이재명 대선 후보 전남 해남 방문 현장에서      이장TV   
1                이재명 골목골목 경청투어[더불어민주당] 전남 강진군 사의재 방문     뉴스프리즘   
2          🔴LIVE 이재명 사의재 방문 - 전남 강진군 / 25.5.11 / KNN  KNN NEWS   
3  [🔴라이브를 켜라] 이재명, 전남 강진군 경청투어…'정약용 유배지' 사의재 방문 /...  채널A News   
4  [생중계] '경청투어' 이재명 민주당 대선 후보, 전남 사의재 방문 (2025.05...     오마이TV   

               pub_date                                          url  
0  2025-05-11T05:31:51Z  https://www.youtube.com/watch?v=hfZ39muIL40  
1  2025-05-11T05:14:03Z  https://www.youtube.com/watch?v=ILz2iRuw9hI  
2  2025-05-11T05:06:30Z  https://www.youtube.com/watch?v=MoYv9m2pk2c  
3  2025-05-11T03:56:16Z  https://www.youtube.com/watch?v=6S5td3jUGmk  
4  2025-05-11T03:57:04Z  https://www.youtube.com/watch?v=vcYoF0eZDBY  

✅ '이재명 전남 방문'에 대한 영상 50건 저장 완료 → youtube_metadata_이재명_전남_방문.csv
